# Model System Following the Moltemplate Approach

## Simulating a box of water using moltemplate and LAMMPS

Here we show an example of a lammps-template file for water. (The
settings shown here are borrowed from the simple-point-charge [8] SPC/E
model.) In addition to coordinates, topology and force-field settings, “LT”
files can optionally include any other kind of LAMMPS settings including
RATTLE or SHAKE constraints, k-space settings, and even group definitions.

In [16]:
%load_ext autoreload
%autoreload 2

import molpy as mp
import numpy as np
from pathlib import Path

data_path = Path("data/case0")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
class SPCE(mp.Struct):

    def __init__(self, name="spce"):
        super().__init__(name=name)
        o = self.def_atom(
            name="o", molid="$w", type="O", q=-0.8476, xyz=[0.00000, 0.00000, 0.00000]
        )
        h1 = self.def_atom(
            name="h1",
            molid="$w",
            type="H",
            q=0.4238,
            xyz=[0.8164904, 0.5773590, 0.00000],
        )
        h2 = self.def_atom(
            name="h2",
            molid="$w",
            type="H",
            q=0.4238,
            xyz=[-0.8164904, 0.5773590, 0.00000],
        )
        self.def_bond(o, h1)
        self.def_bond(o, h2)
        topology = self.get_topology()
        angles = self.gen_angles(topology)
        self.add_angles(angles)

In [18]:
ff = mp.ForceField(name="spce", unit="real")
atomstyle = ff.def_atomstyle("full")
o_type = atomstyle.def_type("O", mass=15.999)
h_type = atomstyle.def_type("H", mass=1.008)

bondstyle = ff.def_bondstyle("harmonic")
bondstyle.def_type(
    o_type, h_type, k=1000.0, r0=1.0
)

anglestyle = ff.def_anglestyle("harmonic")
anglestyle.def_type(
    h_type, o_type, h_type, k=1000.0, theta0=109.47
)

pairstyle = ff.def_pairstyle("lj/charmm/coul/long", inner=9.0, outer=10.0, cutoff=10.0, mix="arithmetic")
pairstyle.def_type(
    o_type, o_type, epsilon=0.1554, sigma=3.1656
)
pairstyle.def_type(
    h_type, h_type, epsilon=0.0, sigma=0.0
)

<PairType: H-H>

In [19]:
typifier = mp.typifier.ForceFieldTypifier(forcefield=ff)
spec = typifier.typify(SPCE())

In [20]:
system = mp.System()
system.set_forcefield(ff)
system.def_box(
    np.diag([31.034, 31.034, 31.034]),
)
for i in range(10):
    for j in range(10):
        for k in range(10):
            system.add_struct(
                spec().move([3.1034 * i, 3.1034 * j, 3.1034 * k]),
            )

mp.io.write_lammps(
    data_path / "spec",
    system.to_frame()
)

## Object composition and coordinate generation

Objects can be connected together to form larger molecule objects. These
objects can be used to form still larger objects. As an example, we define a
small 2-atom molecule named “Monomer”, and use it to construct a short
polymer ("Polymer").

In [21]:
class Monomer(mp.Struct):

    def __init__(self, name):
        super().__init__(name=name)
        ca = self.def_atom(name="ca", molid="$", type="CA", q=0.0, xyz=[0.0, 1.0, 0.0])
        r = self.def_atom(name="r", molid="$", type="R", q=0.0, xyz=[0.0, 4.4, 0.0])
        self.def_bond(ca, r)

In [22]:
class Polymer(mp.Struct):

    def __init__(self, name="polymer"):
        super().__init__(name=name)
        prev = self.add_struct(
            Monomer(name="mon1")
        )
        for i in range(2, 8):
            curr = self.add_struct(
                Monomer(name=f"mon{i}").rotate(180*i, [1, 0, 0]).move([3.2*i, 0, 0])
            )
            self.def_bond(
                prev["atoms"][0],
                curr["atoms"][1],
            )

In [23]:
ff = mp.ForceField(name="polymer", unit="real")
atomstyle = ff.def_atomstyle("full")
ca_type = atomstyle.def_type("CA", mass=13.0)
r_type = atomstyle.def_type("R", mass=50.0)
bondstyle = ff.def_bondstyle("harmonic")
bondstyle.def_type(
    ca_type, r_type, k=15.0, r0=3.4
)
bondstyle.def_type(
    ca_type, ca_type, k=15.0, r0=3.7
)
anglestyle = ff.def_anglestyle("harmonic")
anglestyle.def_type(
    ca_type, r_type, ca_type, k=15.0, theta0=180.0
)
anglestyle.def_type(
    ca_type, ca_type, ca_type, k=15.0, theta0=180.0
)
dihestyle = ff.def_dihedralstyle("charmm")
dihestyle.def_type(
    ca_type, ca_type, ca_type, ca_type, k=15.0, phi0=180.0
)
dihestyle.def_type(
    r_type, ca_type, ca_type, r_type, k=15.0, phi0=180.0
)
pairstyle = ff.def_pairstyle("lj/charmm/coul/long", inner=9.0, outer=10.0, cutoff=10.0, mix="arithmetic")
pairstyle.def_type(
    ca_type, ca_type, epsilon=0.1554, sigma=3.1656
)
pairstyle.def_type(
    ca_type, r_type, epsilon=0.1554, sigma=3.1656
)


<PairType: CA-R>

In [ ]:
system = mp.System()
system.set_forcefield(ff)
for i in range(10):
    system.add_struct(
        Polymer(name="polymer").move([3.2 * i, 0, 0]),
    )

mp.io.write_lammps(
    data_path / "polymer",
    system.to_frame()
)

TypeError: 'System' object is not subscriptable